# Práctica Fútbol. Realizada por Daniel Tomé Gordo

El fichero de datos sobre el que se trabajará es el de DataSetPartidos.txt.

## Segunda tarea: Inicializar spark context y cargar los datos desde el fichero.

In [1]:
from pyspark import SparkContext
sc = SparkContext()

In [2]:
archivo = "./DataSetPartidos.txt"
data = sc.textFile(archivo)

Se realiza un _take_ para ver como es la estructura de los datos. Parece ser que el primer dato es un identificador, el segundo la temporada en la que el partido se juega, el tercero podría ser la jornada en la que se juega el partido, siendo el cuarto y el quinto datos los equipos que jugaron el partido de local y visitante respectivamente. El sexto y séptimo dato son los goles del partido, local y visitante, respectivamente. El octavo es la fecha del partido y el último podría ser la asistencia (**NO SÉ LO QUÉ ES**).
Con el _count_ hallamos los registros totales, 34.518 partidos en este caso

In [6]:
data.take(1)

['1::1970-71::1::Athletic Club::Barcelona::1::1::12/09/1970::21942000.0']

In [7]:
data.count()

34518

Para parsear los datos, se observa que cada variable está dividida por **::**, por lo que se divide cada registro por esos caracteres.

In [9]:
datos_parseados = data.map(lambda x: x.split("::"))

In [19]:
datos_parseados.take(1)

[['1',
  '1970-71',
  '1',
  'Athletic Club',
  'Barcelona',
  '1',
  '1',
  '12/09/1970',
  '21942000.0']]

## Tercera pregunta: Calcular el número total de goles que ha marcado el Real Sporting de Gijón.

En primer lugar se deberán filtrar todos los partidos en los que el Sporting de Gijón juega en casa. Se coge el cuarto elemento de los datos parseados. Una vez filtrados los registros, se coge el sexto elemento (los goles del equipo local) y se suman todos. Así ya se habrían conseguido los goles que el Sporting ha marcado en El Molinón durante las temporadas contenidas en la base de datos. El proceso es el mismo para los goles que el Sporting ha marcado fuera de casa.

Finalmente se "imprimen" los goles del Sporting en esta temporada, sumando ambos resultados.

In [20]:
sporting_local = datos_parseados.filter(lambda x: x[3] == "Sporting de Gijon") # Se filtra por el equipo local (Sporting)
goles_local = sporting_local.map(lambda x: int(x[5])).sum() # Goles marcados en El Molinón

In [21]:
sporting_visitante = datos_parseados.filter(lambda x: x[4] == "Sporting de Gijon") # Se filtra por el equipo visitante
goles_visitante = sporting_visitante.map(lambda x: int(x[6])).sum() # Goles marcados por el Sporting fuera de casa

In [22]:
print ("El Real Sporting de Gijón ha marcado",goles_local + goles_visitante, "goles a lo largo de las 45 temporadas")

El Real Sporting de Gijón ha marcado 2042 goles a lo largo de las 45 temporadas


## Cuarta pregunta: ¿ En que temporada se marcaron más goles?

En primer lugar, se calculan todos los goles que se han marcado en los 34.518 partidos. Para ello, se hallan los goles marcados en los partidos y se guardan en el formato _temporada;goles_. En el último paso, se reducen los registros sumando todos los goles de cada temporada. Para hallar la temporada en la que más goles se han conseguido, se realiza un _takeOrdered_ sacando un registro (el que más valor tenga). Así la salida, será la temporada que más.
Finalmente, se imprime la información.

In [23]:
temporada_goles = datos_parseados.map(lambda x: (x[1], int(x[5]) + int(x[6]))).reduceByKey(lambda a, b: a + b)

In [35]:
temporada_mas_goles = temporada_goles.takeOrdered(1, lambda x: -x[1])

In [36]:
print("La temporada con más goles fue la", temporada_mas_goles[0][0], "y se marcaron", temporada_mas_goles[0][1], "goles")

La temporada con más goles fue la 2012-13 y se marcaron 2294 goles


## Quinta pregunta: ¿Cúal es el equipo y tiene el record de más goles como local? ¿ y cómo visitante?

En primer lugar se sacarán todos los equipos locales de todos los partidos, y en un formato de clave (siendo esta el equipo) - valor (siendo este el número de goles). Posteriormente, se agruparán y se sumarán todos los valores. De esta manera, el que tenga un valor máximo será el equipo que más goles ha metido de local. Para el cálculo del visitante, se realizará el mismo proceso.

In [37]:
goles_totales_local = datos_parseados.map(lambda x: (x[3], int(x[5]))).reduceByKey(lambda a, b: a + b)
mas_gol_local = goles_totales_local.takeOrdered(1, lambda x: -x[1])

In [39]:
print('El equipo con más goles como local a lo largo de estas temporadas es el:', mas_gol_local[0][0], 'con', mas_gol_local[0][1], 'goles')

El equipo con más goles como local a lo largo de estas temporadas es el: Real Madrid con 2054 goles


In [45]:
goles_totales_visitante = datos_parseados.map(lambda x: (x[4], int(x[6]))).reduceByKey(lambda a, b: a + b)
mas_gol_visitante = goles_totales_visitante.takeOrdered(1, lambda x: -x[1])

[('Real Madrid', 1296),
 ('Barcelona', 1296),
 ('Atletico de Madrid', 988),
 ('Valencia', 919),
 ('Betis', 853)]

In [46]:
print('El equipo con más goles como visitante a lo largo de estas temporadas es el:', mas_gol_visitante[0][0], 'con', mas_gol_visitante[0][1], 'goles')

El equipo con más goles como visitante a lo largo de estas temporadas es el: Real Madrid con 1296 goles


## Sexta pregunta: ¿Cúales son las 3 décadas en las que más goles se metieron?

##  Séptima pregunta: ¿Cúal es la media de victorias de los equipos que han estado menos de 10 temporadas en primera división?

##  Octava pregunta: ¿Cúal es la media de goles como visitante por partido?

Se sacarán todos los goles que se han marcado como visitante en los partidos y luego se realizará la media de estos.

In [55]:
total_gol_visitante = datos_parseados.map(lambda x: int(x[6])).sum()
media_goles_visitante = (total_gol_visitante/data.count())
print('Un equipo visitante marca de media', round(media_goles_visitante, 2), 'goles por partido')

Un equipo visitante marca de media 0.93 goles por partido
